In [3]:
import os
import openai
import pandas as pd
import time
import random
import re

# OpenAI API 키 설정
openai.api_key = os.getenv('OPENAI_API_KEY')

# 데이터 경로 설정
input_folder = "C:/fintech_service/final_project/data"
reviews_file = os.path.join(input_folder, "reviews.csv")
stores_file = os.path.join(input_folder, "stores.csv")
output_file = os.path.join(input_folder, "부정리뷰_유사도분석결과.csv")

# 리뷰 데이터와 가게 데이터 불러오기
review_data = pd.read_csv(reviews_file)
stores_data = pd.read_csv(stores_file)

# 사용할 가게 이름 리스트
target_stores = ["깡돈", "다돈식당", "돼지벅스", "사람사는고깃집김일도_뱅뱅점"]

# stores_data에서 대상 가게 필터링
filtered_stores = stores_data[stores_data['store_name'].isin(target_stores)]

# store_id로 review_data와 연결
merged_data = pd.merge(review_data, filtered_stores, on="store_id", how="inner")

# 부정 리뷰만 필터링
negative_reviews = merged_data[(merged_data['final_sentiment'] == "부정적") |
                               (merged_data['taste'] == "부정적") |
                               (merged_data['service'] == "부정적") |
                               (merged_data['quantity'] == "부정적")]

# 부정적인 키워드 및 문장 리스트
negative_sentences = [
    "음식이 별로예요", "양이 적어요", "가성비가 나빠요", "음식이 제대로 조리되지 않았어요", "잡내가 나요",
    "고기 질이 안 좋아요", "재료가 신선하지 않아요", "음식이 너무 맵거나 짜요", "음식이 차갑게 나왔어요",
    "친절하지 않아요", "서비스 응대가 전문적이지 않아요", "예약이 잘 안 돼요", "음식이 너무 늦게 나와요",
    "주문 실수가 잦아요", "직원이 부족해요", "매장이 좁아요", "매장이 지저분해요", "매장이 너무 덥거나 추워요",
    "소음이 심해요", "좌석이 불편해요", "조명이 너무 어두워요", "벌레가 나왔어요", "Wi-Fi가 안 돼요",
    "환기가 잘 안 돼요", "위생 상태가 불량해요", "화장실 상태가 안 좋아요", "특별한 메뉴가 없어요",
    "음료나 주류 품질이 별로예요", "기대보다 평범했어요", "음식 플레이팅이 허술해요", "주차하기 불편해요",
    "교통이 불편해요", "웨이팅이 너무 길어요", "포장이 부실해요", "단체 모임하기 불편해요", "인테리어가 별로예요",
    "숨겨진 추가 비용이 있어요", "쿠폰이나 할인 적용이 잘 안 돼요", "외국인에게 불친절해요", "운영시간이 달라요",
    "결제 수단이 제한적이에요", "음악이 너무 시끄러워요"
]

# GPT-3.5를 사용하여 리뷰와 부정 문장의 유사도 비교
def find_most_similar_sentence(review):
    for attempt in range(3):  # 최대 3번 재시도
        try:
            # OpenAI ChatCompletion 호출
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "당신은 리뷰와 주어진 문장 간의 유사성을 평가하는 전문가입니다."},
                    {"role": "user", "content": f"리뷰: '{review}'"},
                    {"role": "user", "content": "다음 문장들과 리뷰의 유사성을 평가해 주세요:"},
                    {"role": "user", "content": "\n".join([f"{i+1}. {sentence}" for i, sentence in enumerate(negative_sentences)])},
                    {"role": "user", "content": "리뷰와 가장 유사한 문장의 번호와 해당 문장을 알려주세요. 예시: '유사한 문장: 3번, 가성비가 나빠요'."}
                ],
                max_tokens=150,
                temperature=0.2,
            )

            # 응답 텍스트 추출
            response_text = response.choices[0].message["content"].strip()
            
            # 정규 표현식을 사용하여 유사한 문장 추출
            match = re.search(r"유사한 문장:\s*(\d+)번,\s*(.+)", response_text)
            if match:
                similar_idx = int(match.group(1)) - 1
                similar_sentence = negative_sentences[similar_idx]
                return similar_sentence
            else:
                print(f"예기치 않은 응답 형식: {response_text}. 재시도 {attempt + 1}/3")
        except openai.error.OpenAIError as e:
            print(f"OpenAI API 호출 오류: {e}. 재시도 {attempt + 1}/3")
            time.sleep(random.uniform(1, 3))  # 1~3초 대기 후 재시도
        except Exception as e:
            print(f"알 수 없는 오류 발생: {e}. 재시도 {attempt + 1}/3")
            time.sleep(random.uniform(1, 3))

    return "오류 발생 - 유사한 문장 찾기 실패"

# 결과를 저장할 리스트 초기화
results = []

# 부정 리뷰들에 대해 유사한 문장을 찾고 결과 저장
for _, review_row in negative_reviews.iterrows():
    review_text = review_row['review_text']
    if pd.isna(review_text):
        continue

    similar_sentence = find_most_similar_sentence(review_text)

    # 카테고리 판단
    if "음식" in similar_sentence or "양" in similar_sentence:
        category = "음식 관련"
    elif "서비스" in similar_sentence or "직원" in similar_sentence:
        category = "서비스 및 응대 관련"
    elif "매장" in similar_sentence or "환경" in similar_sentence:
        category = "매장 환경 관련"
    elif "위생" in similar_sentence or "화장실" in similar_sentence:
        category = "화장실 및 위생 관련"
    elif "주차" in similar_sentence or "교통" in similar_sentence:
        category = "편의성 관련"
    elif "가격" in similar_sentence or "비용" in similar_sentence:
        category = "가격 및 비용 관련"
    else:
        category = "기타"

    # 결과 저장
    results.append({
        "카테고리": category,
        "유사한 리뷰 내용": review_text,
        "유사한 문장": similar_sentence
    })

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# CSV 파일로 저장
results_df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"결과 저장 완료: {output_file}")


결과 저장 완료: C:/fintech_service/final_project/data\부정리뷰_유사도분석결과.csv
